In [1]:
from scipy import stats as st
import pandas as pd
import math
import numpy as np

In [2]:
des_df = pd.read_excel('data/20220928_design_sheet.xlsx')#('data/LibraryDesign.xls')
nat_df = pd.read_excel('data/LibraryNatural.xls')

filter1 = des_df['count_t0_ex1'] < 5
filter2 = des_df['count_t0_ex2'] < 5

nat_df = nat_df.drop_duplicates(subset='Sequence_aligned')
des_df.RE_norm.loc[filter1|filter2] = np.nan

C:\Users\npral\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


#### Prepare  mmd data:

In [35]:
globalmmd_df = des_df[des_df.header.str.contains('mmd_')]
nat_df = nat_df[nat_df['In Training Set']==1]

# sort the values in terms of norm r.e.
globalmmd_df = globalmmd_df.sort_values('RE_norm')
nat_df = nat_df.sort_values('RE_norm')

# drop any empty values in terms of norm r.e.:
globalmmd_df = globalmmd_df.dropna(subset = ['RE_norm'])
nat_df = nat_df.dropna(subset = ['RE_norm'])

In [36]:
des_df[ des_df.header.str.contains('bmDCA')]

,header,seq,count_t0_ex1,count_typd_ex1,count_t05_ex1,count_t1_ex1,re_ypd_ex1,re_05_ex1,re_1_ex1,count_t0_ex2,...,count_t1_ex2,re_ypd_ex2,re_05_ex2,re_1_ex2,closest_id,Sho1_id,RE_norm,MMD_z0,MMD_z1,MMD_z2
4881,bmDCA9_0,--CIEDVAKYDFEAEEDDHNEMSFVKFEFLRLLDDLDNWWIAKNAL...,124,76,78,8,-0.132350,-0.695585,-2.074529,358,...,26,-0.200193,-0.978654,-2.149135,0.457627,0.372881,-0.173118,-0.044476,0.566748,-0.106230
4882,bmDCA9_1,DEGKTVTARYDYEAAADDDTEQMFPEGAKIIEIEVVSDWWTGVYAS...,120,150,100,12,0.177168,-0.573439,-1.884197,343,...,44,0.002832,-0.817753,-1.902067,0.627119,0.271186,-0.052143,-1.110151,-1.013422,0.891722
4883,bmDCA9_2,GKGRTVEAIYDYEAAEDDDTELGFKEGDIITNIEMVCDWWSGELAR...,67,26,6,1,-0.330843,-1.542182,-2.710272,67,...,4,-0.016453,-0.941042,-2.234240,0.677966,0.271186,-0.379220,-0.927568,-0.837359,0.523748
4884,bmDCA9_3,AAGVTAIAEYDYEAAEDDDNEISFGEGDVIENIEFPDWWWQGRLAN...,30,45,40,2,0.256349,-0.369319,-2.060288,54,...,7,0.122986,-0.652385,-1.897521,0.711864,0.355932,-0.101709,-1.104004,-1.056684,0.738382
4885,bmDCA9_4,GKGYTAVALFDYEAAEDDDNEISFVEGELITQIRFVSDWWQGRYAQ...,129,125,68,15,0.066579,-0.772339,-1.818696,208,...,27,0.267021,-0.786300,-1.896925,0.694915,0.389831,-0.031878,-1.014587,-0.942236,0.541712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8616,bmDCA9_3735,DGILQARALYDYEAQDDDDREISFREGDIIDVITKNHDWW-GECAN...,56,91,38,5,0.291112,-0.662662,-1.933415,106,...,6,0.225109,-0.945297,-2.257380,0.576271,0.423729,-0.161377,-0.319513,-0.266022,0.305220
8617,bmDCA9_3736,AWGRTA-ALYDYEATEDDDEEISFASGEIITNIEMQSDWWQGLDA-...,148,133,102,14,0.033848,-0.655920,-1.908331,323,...,27,0.162418,-0.804626,-2.088064,0.644068,0.372881,-0.108854,-0.998849,-0.803301,0.489695
8618,bmDCA9_3737,GGLITAIAMYDYMAAEDDSREVGFSTGNVITNIEY-ADWWSGLNAN...,198,189,154,35,0.060055,-0.603402,-1.636794,420,...,57,0.096443,-0.732464,-1.877600,0.593220,0.288136,0.025748,-0.745029,-0.608223,0.375807
8619,bmDCA9_3738,QESRVTRAIYDFCGAEDDATELTLREGDEILKIEFDVEWWTGTNAS...,197,157,148,33,-0.018308,-0.618463,-1.660149,366,...,41,0.233906,-0.678449,-1.960923,0.508475,0.186441,-0.003274,-0.288032,-0.327824,0.138549


In [37]:
nat_df.shape[0]

4571

In [39]:

# local mmd sequences
localmmd_df = des_df[des_df.header.str.contains('mmdlocal_')] 
localmmd_df = localmmd_df.sort_values('RE_norm') # sort the values in terms of norm r.e.
localmmd_df = localmmd_df.dropna(subset = ['RE_norm']) # drop any empty valeus in terms of norm r.e

# global VAE sequences
globalVAE_df = des_df[des_df.header.str.contains('VAE_')]
globalVAE_df = globalVAE_df.sort_values('RE_norm') # sort the values in terms of norm r.e.
globalVAE_df = globalVAE_df.dropna(subset = ['RE_norm']) # drop any empty values in terms of norm r.e.

# local VAE sequences
localVAE_df = des_df[des_df.header.str.contains('VAElocal_')]
localVAE_df = localVAE_df.sort_values('RE_norm') # sort the values in terms of norm r.e.
localVAE_df = localVAE_df.dropna(subset = ['RE_norm']) # drop any empty values in terms of norm r.e.

# bmDCA sequences
bmDCA_df = des_df[des_df.header.str.contains('bmDCA')]
bmDCA_df = bmDCA_df.sort_values('RE_norm') # sort the values in terms of norm r.e.
bmDCA_df = bmDCA_df.dropna(subset = ['RE_norm']) # drop any empty values in terms of norm r.e.


In [49]:
localmmd_df.head()

,header,seq,count_t0_ex1,count_typd_ex1,count_t05_ex1,count_t1_ex1,re_ypd_ex1,re_05_ex1,re_1_ex1,count_t0_ex2,...,count_t1_ex2,re_ypd_ex2,re_05_ex2,re_1_ex2,closest_id,Sho1_id,RE_norm,MMD_z0,MMD_z1,MMD_z2
11565,mmdlocal_944,KKQKKGKALYDYTAQSDDDDEVSVAEGDEVVVIDKNKEWWQVRRAN...,79,45,28,1,-0.164156,-0.944727,-2.781824,118,...,8,0.132451,-0.878641,-2.179018,0.694915,0.372881,-0.385084,-1.726624,0.826376,0.100487
10860,mmdlocal_239,RFPCWVKAIYSWGGESDDKRDIGFIEHDLLECLNAGGRWWMGKLAR...,81,69,119,1,0.010622,-0.327196,-2.792682,117,...,10,0.062121,-0.834200,-2.078412,0.796610,0.305085,-0.361304,-1.351802,0.214397,1.446795
11430,mmdlocal_809,KFLYRVEALYSYTAEDDDDREWSMAQGERLEVLNKDGKWWQAKKAD...,108,87,53,2,-0.013646,-0.803406,-2.616591,176,...,11,0.096533,-1.024688,-2.214346,0.711864,0.457627,-0.346883,-1.050940,0.417802,0.622117
11586,mmdlocal_965,TFRYKAKALWSYNANEDDINDISFSKGEILEVDDIDGKWWTAKRAN...,51,26,21,5,-0.212339,-0.879609,-1.892797,89,...,1,-0.033313,-0.817203,-2.959616,0.779661,0.610169,-0.337500,-0.742747,0.483754,0.969615
11501,mmdlocal_880,DYGYKSRALYSYDAQIDDAEEISFSKGEILEVLDMQGGWWQARAAN...,77,77,28,1,0.080258,-0.933591,-2.770688,243,...,26,0.155515,-0.779038,-1.980859,0.694915,0.542373,-0.328868,-0.645100,0.715178,0.877580


In [43]:
bmDCA_df[ bmDCA_df.header.str.contains('bmDCA') ]

,header,seq,count_t0_ex1,count_typd_ex1,count_t05_ex1,count_t1_ex1,re_ypd_ex1,re_05_ex1,re_1_ex1,count_t0_ex2,...,count_t1_ex2,re_ypd_ex2,re_05_ex2,re_1_ex2,closest_id,Sho1_id,RE_norm,MMD_z0,MMD_z1,MMD_z2
8445,bmDCA9_3564,PAVPMVRAMWAFNGEEDDEEDMSFEAGDVIEVNEESNRWWLGKDAR...,104,63,41,1,-0.137435,-0.898507,-2.901231,110,...,3,0.118621,-0.651210,-2.574497,0.576271,0.186441,-0.525303,-0.082063,0.091683,0.688544
6439,bmDCA9_1558,PVPEYVEALYPYDAQEDDHTDLNLRAGEKIEVTVKKKGWWIGMNAV...,72,22,29,1,-0.434652,-0.889193,-2.741530,48,...,1,-0.098863,-0.583134,-2.691467,0.508475,0.305085,-0.510535,0.285845,-0.498031,0.111724
7228,bmDCA9_2347,DSLEMAIFLENYTASDDDKTELSFNEGYVVSIISKDPGWWMGELAK...,72,53,72,1,-0.052798,-0.494258,-2.741530,234,...,5,-0.061539,-0.793987,-2.680472,0.474576,0.254237,-0.507594,-0.159232,-0.127844,0.137265
7960,bmDCA9_3079,AGKIAARALYDYVAGEDDDEEVTFREGDKISDIEKVNDWWKGRTAI...,84,59,30,1,-0.073169,-0.941416,-2.808476,99,...,3,0.195536,-0.788383,-2.528740,0.593220,0.271186,-0.486315,-0.860029,-0.676802,0.283228
6407,bmDCA9_1526,YSYCPTRALYDYTKFDDDDDELSFRAGDSIIVISGLDGWWDITSAT...,44,28,51,1,-0.116036,-0.430141,-2.527650,111,...,2,-0.168754,-0.938442,-2.754519,0.542373,0.338983,-0.465721,-0.200732,0.070876,0.373625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7239,bmDCA9_2358,CVVYFAIAQFTFIGQSDDERDLMFKKGDIIEIVKKKEDKWTGKIAN...,64,37,31,34,-0.157720,-0.809076,-1.158898,128,...,121,-0.348741,-0.690699,-1.034650,0.542373,0.254237,0.389041,0.419889,-0.678711,0.388378
7397,bmDCA9_2516,SLMPTVTAIYFFEYMEDDEFDLSFKMGHKITVIERSDDWWKGRCAN...,87,144,94,53,0.299101,-0.460649,-1.099441,173,...,204,0.210250,-0.560622,-0.938642,0.508475,0.254237,0.431868,0.406697,-0.537926,0.481415
5802,bmDCA9_921,QIHETVQALYDYNAFNDDDDELSFREGDRLFVVDKHGEWWENRNAN...,47,59,49,31,0.179012,-0.476160,-1.064933,92,...,109,0.122040,-0.839873,-0.936587,0.542373,0.389831,0.442369,-0.588370,0.406354,0.138192
8603,bmDCA9_3722,DPGPTAQALYDYEAAEDDDNEISFAEGE-ITDIEEVDGWWEGAAAK...,98,130,91,83,0.202976,-0.526443,-0.956345,198,...,228,0.262006,-0.698422,-0.948956,0.711864,0.355932,0.470376,-1.139330,-1.072722,0.456000


In [40]:
num_globalmmd, num_localmmd, num_nat, num_globalVAE, num_localVAE, num_bmDCA = globalmmd_df.shape[0], \
                         localmmd_df.shape[0], \
                         nat_df.shape[0], \
                         globalVAE_df.shape[0], \
                         localVAE_df.shape[0], \
                         bmDCA_df.shape[0]


print(f'Global MMD: {num_globalmmd}, local MMD: {num_localmmd}, natural: {num_nat}, global VAE: {num_globalVAE},' +
     f'local VAE: {num_localVAE}, bmDCA: {num_bmDCA}')

Global MMD: 1890, local MMD: 955, natural: 4571, global VAE: 3756,local VAE: 858, bmDCA: 3572


In [24]:
""" The null hypothesis for 2-sample test is rejected for large samples from N and M distributions. In particular,

D < c(alpha) sqrt( (n+m)/(n*m) ), where c(alpha) and alpha is found from a lookup table and is choosen depending on 
the degree of stringency. For example, higher alpha value means easier to not reject the null hypothesis, while lower
alpha means it is easier to reject the null hypothesis. Usually, alpha = 0.05 is choosen, so c(alpha) is 1.358 (ref: https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test).
"""

def check_nullH_2smaple(D, alpha, n, m):
    """
    function:
    
    arg:
    
    return:
    
    """
    c = {0.20:1.073, 0.15:1.138, 0.10:1.224, 0.05:1.358, 0.025:1.48, 0.01:1.628, 0.005:1.731, 0.001:1.949}
    c_alpha = c[alpha]
    null_threshold = c_alpha * math.sqrt( (n+m)/ (n*m))
    if D < null_threshold:
        print('Null hypothesis cannot be rejected')
        decision = 'Keep'
    else:
        print('Reject null hypothesis')
        decision = 'Reject'
    
    return decision

## KS 2 smaple test between design and natural homolog sequences:

### Global mmd sequences

In [53]:
global_normre = globalmmd_df.RE_norm.values

# norm re scores
nat_normre = nat_df.RE_norm.values

# compute ks test for 2 sample 
twosample_re = st.ks_2samp(global_normre, nat_normre, 'two-sided')


# ks statistics and p value
D_2test = twosample_re[0]
p_2test = twosample_re[1]

# compute if the null hypothesis is rejected or not. 
alpha = 0.05
decision_2smaple = check_nullH_2smaple(D_2test, alpha, len(global_normre), len(nat_normre))

print('p = ', p_2test)

Null hypothesis cannot be rejected
p =  0.05685965652974645


### Local mmd sequences

In [54]:
local_normre = localmmd_df.RE_norm.values

# norm re scores
nat_normre = nat_df.RE_norm.values

# compute ks test for 2 sample 
twosample_re_localMMD = st.ks_2samp(local_normre, nat_normre, 'two-sided')


# ks statistics and p value
D_2test = twosample_re_localMMD[0]
p_2test = twosample_re_localMMD[1]

# compute if the null hypothesis is rejected or not. 
alpha = 0.05
decision_2smaple = check_nullH_2smaple(D_2test, alpha, len(local_normre), len(nat_normre))

print('p = ', p_2test)

Reject null hypothesis
p =  9.063653148156047e-172


### Global VAE sequences

In [55]:
globalVAE_normre = globalVAE_df.RE_norm.values

# norm re scores
nat_normre = nat_df.RE_norm.values

# compute ks test for 2 sample 
twosample_re_globalVAE = st.ks_2samp(globalVAE_normre, nat_normre, 'two-sided')


# ks statistics and p value
D_2test = twosample_re_globalVAE[0]
p_2test = twosample_re_globalVAE[1]

# compute if the null hypothesis is rejected or not. 
alpha = 0.05
decision_2smaple = check_nullH_2smaple(D_2test, alpha, len(globalVAE_normre), len(nat_normre))

print('p = ', p_2test)

Reject null hypothesis
p =  0.00012211449518184647


### Local VAE sequences

In [56]:
localVAE_normre = localVAE_df.RE_norm.values

# norm re scores
nat_normre = nat_df.RE_norm.values

# compute ks test for 2 sample 
twosample_re_localVAE = st.ks_2samp(localVAE_normre, nat_normre, 'two-sided')


# ks statistics and p value
D_2test = twosample_re_localVAE[0]
p_2test = twosample_re_localVAE[1]

# compute if the null hypothesis is rejected or not. 
alpha = 0.05
decision_2smaple = check_nullH_2smaple(D_2test, alpha, len(localVAE_normre), len(nat_normre))

print('p = ', p_2test)

Reject null hypothesis
p =  1.2212453270876722e-15


### bmDCA:

In [57]:
bmDCA_normre = bmDCA_df.RE_norm.values

# norm re scores
nat_normre = nat_df.RE_norm.values

# compute ks test for 2 sample 
twosample_re_bmDCA = st.ks_2samp(bmDCA_normre, nat_normre, 'two-sided')


# ks statistics and p value
D_2test = twosample_re_bmDCA[0]
p_2test = twosample_re_bmDCA[1]

# compute if the null hypothesis is rejected or not. 
alpha = 0.05
decision_2smaple = check_nullH_2smaple(D_2test, alpha, len(bmDCA_normre), len(nat_normre))

print('p = ', p_2test)

Reject null hypothesis
p =  7.035081874651183e-05


# One-sample KS tests (sanity checks):

In [7]:
# compute 1sample test and compare if the global design sequences are normally distribution 
onesample_global = st.kstest(global_normre, 'norm')

# ks statistics and p value
D_1test_globalmmd = onesample_global[0]
p_1test_globalmmd = onesample_global[1]

print(f'Since the p value {p_1test_globalmmd} < 0.05, the null hypothesis can be rejected. Therefore, the distribution is not normally distributed.')

Since the p value 9.389750969076114e-244 < 0.05, the null hypothesis can be rejected. Therefore, the distribution is not normally distributed.


In [8]:
# compute 1sample test and compare if the natural sequences are normally distribution 
onesample_nat = st.kstest(nat_normre, 'norm')

# ks statistics and p value
D_1test_nat = onesample_nat[0]
p_1test_nat = onesample_nat[1]

print(f'Since the p value {p_1test_nat} < 0.05, the null hypothesis can be rejected. Therefore, the distribution is not normally distributed.')

Since the p value 0.0 < 0.05, the null hypothesis can be rejected. Therefore, the distribution is not normally distributed.
